In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [2]:
df = pd.read_csv('final_features.csv')

In [3]:
df.reset_index(drop = True,inplace = True)
df.drop(df.columns[0],axis=1 , inplace = True)
df

,product_id,seller_index,category_index,review_score,order_item_id
0,00066f42aeeb9f3007548bb9d3f33c38,2,2.9,5.0,1
1,00088930e925c41fd95ebfe695fd2655,2,14.5,4.0,0
2,0009406fd7479715e4bef61dd91f2462,2,16.3,1.0,0
3,000b8f95fcb9e0096488278317764d19,2,16.9,5.0,2
4,000d9be29b5207b54e86aa1b1ac54872,2,2.8,5.0,0
...,...,...,...,...,...
32946,fff6177642830a9a94a0f2cba5e476d1,2,2.1,4.5,0
32947,fff81cc3158d2725c0655ab9ba0f712c,2,2.7,4.0,0
32948,fff9553ac224cec9d15d49f5a263411f,2,5.2,5.0,0
32949,fffdb2d0ec8d6a61f0a0a0db3f25b441,2,12.2,5.0,0


In [4]:
vector = df[['seller_index','category_index','review_score']].values
item_ids = df.index
product_id = df.product_id

In [5]:
def get_product(item_id):
    idx = item_ids[item_id]
    item_profile = vector[idx:idx+1]
    return item_profile
def get_related_products(ids):
    item_profile_list = [get_product(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profile_list)
    return item_profiles

In [6]:
class ContentBasedRecommender:
    MODEL_NAME = 'Content-Based'
    def __init__(self,item_ids=None):
        self.item_ids = item_ids
    def get_model_name(self):
        return self.MODEL_NAME
    def _get_similar_products(self, item_id, topn=1000):
        cosine_similarities = cosine_similarity(get_product(item_id),vector)
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_products = sorted([(item_ids[i],cosine_similarities[0,i]) for i in similar_indices ], key=lambda x : -x[1])
        return similar_products
    def get_recommendation(self,item_id,items_to_ignore = [], topn=10, verbose = False):
        items_to_ignore.append(item_id)
        similar_items = self._get_similar_products(item_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        return similar_items_filtered[:10]

In [7]:
csm = ContentBasedRecommender(item_ids)

In [8]:
x = csm.get_recommendation(23431)[:5]
x


[(252, 1.0), (29567, 1.0), (29409, 1.0), (24470, 1.0), (20699, 1.0)]

In [9]:
ids = []
for i in range(0,len(x)):
    ids.append(x[i][0])
pids = []
for i in range (0,len(ids)):
    pids.append(product_id[ids[i]])
pids


['01e20e6604216c8adb31d463214ba00c',
 'e5ac4f7201729a0d3639d979c104b91d',
 'e4835fa9d35d4226f23380c4a7cf12ec',
 'be751484da30a0a6f7ded2b640ee091c',
 'a0ff95d002b9e836b60b47e97a200f93']

In [11]:
df1 = pd.read_csv('database_f.csv')


In [15]:
df1[df1.product_id=="e5ac4f7201729a0d3639d979c104b91d"]

,product_id,product_name,product_category_name_english,product_weight_g,product_length_cm,product_height_cm,product_width_cm,price,review_score
23936,e5ac4f7201729a0d3639d979c104b91d,home_construction Product 28,home_construction,2600.0,38.0,28.0,28.0,239.9,5.0
